In [2]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import pandas as pd
import numpy as np

from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.error import URLError
from tqdm import tqdm
import re
import json

# OWGR Tournament Gathering

In [3]:
def process_tournament_owgr(tournament_id):
    tournament_id = str(tournament_id)
    url = "https://www.owgr.com/events/" + tournament_id

    try:
        # get the Scores:

        html = urlopen(url)
        soup = BeautifulSoup(html, 'lxml')
        
        # Find the table
        table = soup.find('table')

        # Get all rows from the table
        rows = table.find_all('tr')

        # Initialize a list to store table data
        table_data = []

        # Iterate through each row
        for row in rows:
            # Get all cells (both headers and regular data)
            cells = row.find_all(['th', 'td'])
            
            # Initialize a list to store row data
            row_data = []
            
            # Iterate through each cell
            for cell in cells:
                # Check if the cell contains a link (for player profile)
                a_tag = cell.find('a', href=True)
                if a_tag:
                    # Extract the href attribute (player profile link)
                    player_id = a_tag['href'].split('-')[-1]  # Extract player ID (e.g., 'greg-dalziel-28451')
                    row_data.append(player_id)    
                       
                # adding data
                text = cell.get_text(strip=True)

                # this is special case for first row, want to add in player ids, which we do above by searching a tags
                if text == "NAME":
                    row_data.append("player_id")

                # another special case when bonus points are awarded
                if "Bonus Points" in text:
                    row_data.append("")

                # always append the text
                row_data.append(text)
            
            # Add non-empty row data to the table_data list
            if row_data:
                table_data.append(row_data)

        # Get metadata:
        
        # Dictionary to store the event information
        event_info = {}

        # Find the div containing the week info (span with the specific classes)
        week_spans = soup.find_all('span', class_='boldedString_content__bolded__QAKW8')
        if week_spans and len(week_spans) >= 2:
            week_number = week_spans[0].get_text(strip=True)
            week_value = week_spans[1].get_text(strip=True)
            event_info['Week'] = f"{week_number} {week_value}"

        # Find the exact event date by targeting spans with specific text pattern
        date_spans = soup.find_all('span', class_='boldedString_content__normal__NreGx')
        if date_spans and len(date_spans) >= 2:
            # Ensuring that only the relevant spans are captured (month/day/year)
            event_date = ' '.join([span.get_text(strip=True) for span in date_spans[:3]])
            event_info['Event Date'] = event_date

        # Find the div with event title (event name)
        event_title = soup.find('div', class_='eventTitleComponent_name__C2ZKJ')
        if event_title:
            event_info['Event Title'] = event_title.get_text(strip=True)

        # Find the div with the field rating
        field_rating = soup.find('div', class_='eventTitleComponent_rating__4UcQg')
        if field_rating:
            event_info['Field Rating'] = re.search(r'\d+\.\d+', field_rating.get_text(strip=True)).group()

        event_info["tournament_id"] = tournament_id
        # tour information

        # Find the script tag with the specific id and type
        script_tag = soup.find('script', {'id': '__NEXT_DATA__', 'type': 'application/json'})

        # Extract the content inside the tag (which is a JSON string)
        json_data = script_tag.string

        # Parse the JSON string into a Python dictionary
        data = json.loads(json_data)

        tour_info = data["props"]["pageProps"]["eventDetailsData"]["eventDetails"]["tours"][0]
        
        merged_dict = dict(event_info, **tour_info)

        scores_df = pd.DataFrame(table_data[1:], columns=table_data[0])
        scores_df["tournament_id"] = tournament_id

        return scores_df, pd.DataFrame([merged_dict])
    
    except URLError as e:
        print(f"Error opening URL for tournament {tournament_id}: {e}")
        return pd.DataFrame(), pd.DataFrame()  # Return an empty dataframe on error
    
    except Exception as e:
        print(f"Error processing tournament {tournament_id}: {e}")
        return pd.DataFrame(), pd.DataFrame()  # Return an empty dataframe on any other error

def process_tournaments(tournament_ids):
    all_table_data = []
    all_merged_data = []

    # Using ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(process_tournament_owgr, tid): tid for tid in tournament_ids}

        # Use tqdm to show progress
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing tournaments"):
            try:
                table_data, merged_data = future.result()
                all_table_data.append(table_data)
                all_merged_data.append(merged_data)
            except Exception as e:
                print(f"Error processing tournament: {e}")

    # Concatenate all DataFrames collected
    concatenated_table_data = pd.concat(all_table_data, ignore_index=True) if all_table_data else pd.DataFrame()
    concatenated_merged_data = pd.concat(all_merged_data, ignore_index=True) if all_merged_data else pd.DataFrame()

    return concatenated_table_data, concatenated_merged_data

In [4]:
tournament_ids = list(range(1, 10609))  # All tournaments as of October 6, 2024

tournament_scores, metadata = process_tournaments(tournament_ids)
tournament_scores

Processing tournaments:  13%|█▎        | 1430/10608 [04:15<24:15,  6.30it/s]  

Error opening URL for tournament 1434: HTTP Error 500: Internal Server Error


Processing tournaments:  21%|██        | 2192/10608 [06:37<18:33,  7.56it/s]  

Error opening URL for tournament 2195: HTTP Error 500: Internal Server Error


Processing tournaments:  24%|██▍       | 2559/10608 [07:49<37:01,  3.62it/s]

...

Error opening URL for tournament 10564: HTTP Error 500: Internal Server Error


Processing tournaments: 100%|█████████▉| 10575/10608 [36:09<00:09,  3.50it/s]

Error opening URL for tournament 10577: HTTP Error 500: Internal Server Error


Processing tournaments: 100%|██████████| 10608/10608 [36:18<00:00,  4.87it/s]


,Finish Pos.,CTRY,player_id,NAME,POINTS WON,RANK FROM,RANK TO,tournament_id,R1,R2,R3,R4,AGG,R5
0,1,,449,Mark McNulty,10.00000,-,-,7,NaN,NaN,NaN,NaN,NaN,NaN
1,2,,440,David Frost,6.00000,-,-,7,NaN,NaN,NaN,NaN,NaN,NaN
2,3,,446,Tony Johnstone,4.00000,-,-,7,NaN,NaN,NaN,NaN,NaN,NaN
3,T4,,95,Rick Hartmann,4.00000,-,-,7,NaN,NaN,NaN,NaN,NaN,NaN
4,T4,,265,David Feherty,4.00000,-,-,7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1138859,MC,,24639,Thabiso Ngcobo,0.00000,2728,2735,10608,81,79,-,NaN,160,NaN
1138860,MC,,22040,Steven Le Roux,0.00000,2811,2815,10608,83,78,-,NaN,161,NaN
1138861,MC,,27951,Ricco Motsa,0.00000,4606,4603,10608,86,78,-,NaN,164,NaN
1138862,MC,,16894,Terence Boardman,0.00000,4031,4027,10608,84,81,-,NaN,165,NaN


# Gathering LIV Data

In [18]:
def process_tournament_liv(tournament_id):
    tournament_id = str(tournament_id)
    url = "https://web-common.livgolf.com/api/leaderboard/players/" + tournament_id

    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    
    # Extract the text from the <p> tag containing the JSON-like content
    json_text = soup.find('p').get_text()

    # Convert the JSON text to a Python dictionary
    data = json.loads(json_text)

    player_results = []

    for player_dict in data["players"]:
        # List of keys to keep
        keys_to_keep = ['id', 'name', 'position', 'totalScore']

        # Create a new dictionary with only the keys you want
        new_d = {key: player_dict[key] for key in keys_to_keep if key in player_dict}

        new_d['player_id_liv'] = new_d.pop('id')

        # Add round scores as separate keys in the new dictionary
        for idx, round_info in enumerate(player_dict['rounds'], start=1):
            new_d[f'R{idx}_topar'] = round_info['score']

        player_results.append(new_d)

    tournament_results = pd.DataFrame(player_results)
    
    tournament_results["tournament_id_liv"] = tournament_id
    
    # gather tournament par information
    url = "https://web-common.livgolf.com/api/leaderboard/courses/" + tournament_id

    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')

    # Extract the text from the <p> tag containing the JSON-like content
    json_text = soup.find('p').get_text()

    # Convert the JSON text to a Python dictionary
    data = json.loads(json_text)

    round_par = int(data["rounds"][0]["totalPar"])
    
    for key in ["R1", "R2", "R3"]:
        tournament_results[f"{key}_topar"] = tournament_results[f"{key}_topar"].replace('E', '0')
        tournament_results[f"{key}_topar"] = np.where(tournament_results[f"{key}_topar"] == '',
                                                            np.nan,
                                                            tournament_results[f"{key}_topar"])

        tournament_results[f"{key}_topar"] = tournament_results[f"{key}_topar"].astype(float)
        tournament_results[key] = tournament_results[f"{key}_topar"] + round_par

    tournament_results["AGG"] = tournament_results["R1"] + tournament_results["R2"] + tournament_results["R3"]
    return tournament_results

def process_tournaments_liv(tournament_ids):
    all_table_data = []

    # Using ThreadPoolExecutor for multithreading
    with ThreadPoolExecutor(max_workers=8) as executor:
        futures = {executor.submit(process_tournament_liv, tid): tid for tid in tournament_ids}

        # Use tqdm to show progress
        for future in tqdm(as_completed(futures), total=len(futures), desc="Processing tournaments"):
            try:
                table_data = future.result()
                all_table_data.append(table_data)
            except Exception as e:
                print(f"Error processing tournament: {e}")

    # Concatenate all DataFrames collected
    concatenated_table_data = pd.concat(all_table_data, ignore_index=True) if all_table_data else pd.DataFrame()

    return concatenated_table_data

# for LIV you can get all the tournaments from one year at once
def get_tournament_info_liv(year):
    year = str(year)
    url = "https://web-common.livgolf.com/api/events/" + year

    html = urlopen(url)
    soup = BeautifulSoup(html, 'lxml')
    
    # Extract the text from the <p> tag containing the JSON-like content
    json_text = soup.find('p').get_text()

    # Convert the JSON text to a Python dictionary
    data = json.loads(json_text)

    tournaments = []

    for tournament_dict in data["events"]:
        # List of keys to keep
        keys_to_keep = ['eventId', 'eventLivId', 'eventTitle', 'endDate']

        # Create a new dictionary with only the keys you want
        new_d = {key: tournament_dict[key] for key in keys_to_keep if key in tournament_dict}

        tournaments.append(new_d)

    tournament_info = pd.DataFrame(tournaments)
    
    return tournament_info

In [19]:
# gathering all liv tournaments for 2022 - 2024 excluing events 8 and 37 which are some team championships and 23 which is a promotion event
tournament_ids = list(range(1, 37))
tournament_ids.remove(8)
tournament_ids.remove(21)
tournament_ids.remove(23)

liv_results = process_tournaments_liv(tournament_ids)
liv_results

Processing tournaments: 100%|██████████| 33/33 [00:02<00:00, 12.27it/s]


,name,position,totalScore,player_id_liv,R1_topar,R2_topar,R3_topar,tournament_id_liv,R1,R2,R3,AGG,R4_topar
0,Eugenio Chacarra,1.0,-19,57,-7.0,-9.0,-3.0,6,65.0,63.0,69.0,197.0,NaN
1,Patrick Reed,2.0,-16,54,-4.0,-7.0,-5.0,6,68.0,65.0,67.0,200.0,NaN
2,Paul Casey,3.0,-15,58,-1.0,-7.0,-7.0,6,71.0,65.0,65.0,201.0,NaN
3,Richard Bland,4.0,-15,2,-7.0,-4.0,-4.0,6,65.0,68.0,68.0,201.0,NaN
4,Sihwan Kim,5.0,-15,17,-5.0,-6.0,-4.0,6,67.0,66.0,68.0,201.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,Mito Pereira,50.0,+5,74,3.0,1.0,1.0,36,73.0,71.0,71.0,215.0,NaN
1667,Jason Kokrak,51.0,+6,60,3.0,3.0,0.0,36,73.0,73.0,70.0,216.0,NaN
1668,Cameron Smith,52.0,+10,63,3.0,3.0,4.0,36,73.0,73.0,74.0,220.0,NaN
1669,Hudson Swafford,53.0,+13,36,8.0,8.0,-3.0,36,78.0,78.0,67.0,223.0,NaN


In [20]:
years = [2022, 2023, 2024]

liv_info = []
for year in years:
    liv_info.append(get_tournament_info_liv(year))

liv_info_df = pd.concat(liv_info)
liv_info_df["eventId"] = liv_info_df["eventId"].astype(int)

# remove team championships
liv_info_df = liv_info_df[liv_info_df["eventId"] != 21]
liv_info_df = liv_info_df[liv_info_df["eventId"] != 37]

liv_info_df = liv_info_df.sort_values(by="eventId").reset_index(drop=True)
liv_info_df

,eventId,eventLivId,eventTitle,endDate
0,1,None,London,2022-06-11T18:00:00.000+01:00
1,2,None,Portland,2022-07-02T19:00:00.000-05:00
2,3,None,Bedminster,2022-07-31T19:00:00.000-06:00
3,4,None,Boston,2022-09-04T19:00:00.000-05:00
4,5,None,Chicago,2022-09-18T19:00:00.000-06:00
5,6,None,Bangkok,2022-10-09T18:00:00.000+07:00
6,7,None,Jeddah,2022-10-16T18:00:00.000+03:00
7,9,None,Mayakoba,2023-02-26T23:59:00.000-05:00
8,10,None,Tucson,2023-03-19T18:00:00.000-07:00
9,11,None,Orlando,2023-04-02T21:00:00.000-05:00


# Merge the datasets

In [84]:
liv_names = liv_results[["name", "player_id_liv"]].drop_duplicates("name")
liv_names["player_id_liv"] = liv_names["player_id_liv"].astype(int)
liv_names = liv_names.sort_values("player_id_liv").reset_index(drop=True)

# in dataset some names have a trailing space and two names are different
def fix_name_liv(name):
    if name[-1] == " ":
        return name[:-1]
    # special cases
    elif name == "Mito Pereira":
        return "Guillermo Mito Pereira"
    elif name == "Ratchanon Chantananuwat":
        return "Ratchanon TK Chantananuwat(Am)"
    else:
        return name

liv_names["NAME"] = liv_names["name"].apply(lambda x: fix_name_liv(x))
liv_names

# gather owgr names
pga_names = tournament_scores[["NAME", "player_id"]].drop_duplicates("NAME")

# bonus points get added as a name and mess things up
pga_names = pga_names[pga_names["player_id"] != ""]

pga_names["player_id"] = pga_names["player_id"].astype(int)
pga_names = pga_names.sort_values("player_id").reset_index(drop=True)

df_merged = liv_names.merge(pga_names, how="inner", on="NAME")

# check that all are matches
assert len(df_merged) == len(liv_names)

df_merged

,name,player_id_liv,NAME,player_id
0,Oliver Bekker,1,Oliver Bekker,12819
1,Richard Bland,2,Richard Bland,6516
2,Laurie Canter,3,Laurie Canter,14795
3,Ratchanon Chantananuwat,4,Ratchanon TK Chantananuwat(Am),27275
4,Hennie Du Plessis,5,Hennie Du Plessis,19421
...,...,...,...,...
80,Caleb Surratt,149,Caleb Surratt,30463
81,Lucas Herbert,150,Lucas Herbert,17064
82,Anthony Kim,151,Anthony Kim,11641
83,Adrian Meronk,152,Adrian Meronk,14196


In [89]:
name_dict

{'Oliver Bekker': 'Oliver Bekker',
 'Richard Bland': 'Richard Bland',
 'Laurie Canter': 'Laurie Canter',
 'Ratchanon Chantananuwat': 'Ratchanon TK Chantananuwat(Am)',
 'Hennie Du Plessis': 'Hennie Du Plessis',
 'Oliver Fisher': 'Oliver Fisher',
 'Sergio Garcia': 'Sergio Garcia',
 'Talor Gooch': 'Talor Gooch',
 'Branden Grace': 'Branden Grace',
 'Justin Harding': 'Justin Harding',
 'Sam Horsfield': 'Sam Horsfield',
 'Dustin Johnson ': 'Dustin Johnson',
 'Matt Jones': 'Matt Jones',
 'Sadom Kaewkanjana': 'Sadom Kaewkanjana',
 'Martin Kaymer': 'Martin Kaymer',
 'Phachara Khongwatmai': 'Phachara Khongwatmai',
 'Sihwan Kim': 'Sihwan Kim',
 'Ryosuke Kinoshita': 'Ryosuke Kinoshita',
 'Chase Koepka': 'Chase Koepka',
 'Jinichiro Kozuma': 'Jinichiro Kozuma',
 'Pablo Larrazabal': 'Pablo Larrazabal',
 'Graeme McDowell ': 'Graeme McDowell',
 'Jediah Morgan': 'Jediah Morgan',
 'Kevin Na': 'Kevin Na',
 'Shaun Norris': 'Shaun Norris',
 'Andy Ogletree': 'Andy Ogletree',
 'Louis Oosthuizen ': 'Louis Oost

In [88]:
liv_results.name.values[0]

'Eugenio Chacarra'

In [187]:
name_dict = dict(zip(df_merged["name"], df_merged["NAME"]))
id_dict = dict(zip(df_merged["player_id_liv"], df_merged["player_id"]))

liv_results["player_id_liv"] = liv_results["player_id_liv"].astype(int)
liv_results["player_id"] = liv_results["player_id_liv"].map(id_dict)
liv_results["NAME"] = liv_results["name"].map(name_dict)

liv_results["tournament_id"] = liv_results["tournament_id_liv"] + "_liv"

# Function to calculate 'Finish Pos', handling NaN values as 'WD'
def calculate_finish_pos(df):
    # Convert NaN in 'position' to 'WD' for withdrawals
    df['Finish Pos.'] = df['position'].apply(lambda x: 'WD' if pd.isna(x) else str(int(x)))
    
    # Find ties (ignoring NaN/withdrawals)
    tied_positions = df[df['Finish Pos'] != 'WD'].groupby('AGG').filter(lambda x: len(x) > 1)
    
    # Process ties
    for agg_value in tied_positions['AGG'].unique():
        tied_rows = df[(df['AGG'] == agg_value) & (df['Finish Pos'] != 'WD')]
        min_pos = tied_rows['position'].min()
        tied_finish = f'T{int(min_pos)}'
        df.loc[(df['AGG'] == agg_value) & (df['Finish Pos'] != 'WD'), 'Finish Pos.'] = tied_finish
    
    return df

# Function to group by 'tournament_id', apply finish pos calculation, and re-aggregate
def fix_finish_pos_liv(df):
    # Apply the finish position calculation for each tournament
    return df.groupby('tournament_id', group_keys=False).apply(calculate_finish_pos).reset_index(drop=True)

liv_results = fix_finish_pos_liv(liv_results)
liv_results

/var/folders/76/xwx2gdv102jgvj2y2l_m0n8w0000gn/T/ipykernel_1658/939438895.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return df.groupby('tournament_id', group_keys=False).apply(calculate_finish_pos).reset_index(drop=True)


,name,position,totalScore,player_id_liv,R1_topar,R2_topar,R3_topar,tournament_id_liv,R1,R2,R3,AGG,R4_topar,player_id,NAME,tournament_id,Finish Pos,Finish Pos.
0,Eugenio Chacarra,1.0,-19,57,-7.0,-9.0,-3.0,6,65.0,63.0,69.0,197.0,NaN,26745,Eugenio Chacarra,6_liv,T1,1
1,Patrick Reed,2.0,-16,54,-4.0,-7.0,-5.0,6,68.0,65.0,67.0,200.0,NaN,14838,Patrick Reed,6_liv,T1,2
2,Paul Casey,3.0,-15,58,-1.0,-7.0,-7.0,6,71.0,65.0,65.0,201.0,NaN,7108,Paul Casey,6_liv,T1,T3
3,Richard Bland,4.0,-15,2,-7.0,-4.0,-4.0,6,65.0,68.0,68.0,201.0,NaN,6516,Richard Bland,6_liv,T1,T3
4,Sihwan Kim,5.0,-15,17,-5.0,-6.0,-4.0,6,67.0,66.0,68.0,201.0,NaN,13307,Sihwan Kim,6_liv,T1,T3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,Mito Pereira,50.0,+5,74,3.0,1.0,1.0,36,73.0,71.0,71.0,215.0,NaN,19455,Guillermo Mito Pereira,36_liv,T11,T47
1667,Jason Kokrak,51.0,+6,60,3.0,3.0,0.0,36,73.0,73.0,70.0,216.0,NaN,12337,Jason Kokrak,36_liv,T16,51
1668,Cameron Smith,52.0,+10,63,3.0,3.0,4.0,36,73.0,73.0,74.0,220.0,NaN,15856,Cameron Smith,36_liv,T26,52
1669,Hudson Swafford,53.0,+13,36,8.0,8.0,-3.0,36,78.0,78.0,67.0,223.0,NaN,14706,Hudson Swafford,36_liv,T35,53


In [188]:
df_combined = pd.concat([tournament_scores, liv_results[["NAME", "player_id", "R1", "R2", "R3", "AGG", "tournament_id", "Finish Pos."]]])

df_combined["points_won"] = df_combined["POINTS WON"].astype(float)
df_combined["start_rank"] = pd.to_numeric(df_combined['RANK FROM'].replace('-', np.nan), errors='coerce')
df_combined["end_rank"] = pd.to_numeric(df_combined['RANK TO'].replace('-', np.nan), errors='coerce')

df_combined = df_combined[["player_id", "NAME", "Finish Pos.", "points_won", "start_rank", "end_rank", "tournament_id", "R1", "R2", "R3", "R4", "R5", "AGG"]].rename(columns={"NAME": "name", "Finish Pos.": "position", "AGG": "total"})
df_combined

,player_id,name,position,points_won,start_rank,end_rank,tournament_id,R1,R2,R3,R4,R5,total
0,449,Mark McNulty,1,10.0,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
1,440,David Frost,2,6.0,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
2,446,Tony Johnstone,3,4.0,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
3,95,Rick Hartmann,T4,4.0,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
4,265,David Feherty,T4,4.0,NaN,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1666,19455,Guillermo Mito Pereira,T47,NaN,NaN,NaN,36_liv,73.0,71.0,71.0,NaN,NaN,215.0
1667,12337,Jason Kokrak,51,NaN,NaN,NaN,36_liv,73.0,73.0,70.0,NaN,NaN,216.0
1668,15856,Cameron Smith,52,NaN,NaN,NaN,36_liv,73.0,73.0,74.0,NaN,NaN,220.0
1669,14706,Hudson Swafford,53,NaN,NaN,NaN,36_liv,78.0,78.0,67.0,NaN,NaN,223.0


In [193]:
metadata["date"] = pd.to_datetime(metadata["Event Date"])
metadata["date"] = pd.to_datetime(metadata["date"])

metadata["week"] = metadata["date"].dt.isocalendar().week
metadata["year"] = metadata["date"].dt.isocalendar().year

metadata["date"] = metadata["date"].dt.date

# metadata['week'] = metadata['Week'].str.extract('(\d+)').astype(int)

liv_info_df["date"] = pd.to_datetime(liv_info_df["endDate"], format='%Y-%m-%dT%H:%M:%S.%f%z', utc=True)
liv_info_df["week"] = liv_info_df["date"].dt.isocalendar().week
liv_info_df["year"] = liv_info_df["date"].dt.isocalendar().year

liv_info_df["date"] = liv_info_df["date"].dt.date

liv_info_df["tournament_id"] = liv_info_df["eventId"].astype(str) + "_liv"
liv_info_df["Event Title"] = liv_info_df["eventTitle"] + "_liv"
liv_info_df["name"] = "Liv Golf"
liv_info_df["tourId"] = metadata.tourId.max() + 1
liv_info_df["code"] = "liv"
liv_info_df["isEligible"] = False

events = pd.concat([metadata[["week", "year", "date", "Event Title", "Field Rating", "tournament_id", "tourId", "name", "code", "isEligible"]], liv_info_df[["date", "week", "year", "Event Title", "tournament_id", "tourId", "name", "code", "isEligible"]]])
events = events.rename(columns={"Event Title": "event", "Field Rating": "field_rating", "tourId": "tour_id", "name": "tour_name", "isEligible": "is_eligible", "code": "tour_code"})
events

/var/folders/76/xwx2gdv102jgvj2y2l_m0n8w0000gn/T/ipykernel_1658/1150902869.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  metadata["date"] = pd.to_datetime(metadata["Event Date"])


,week,year,date,event,field_rating,tournament_id,tour_id,tour_name,tour_code,is_eligible
0,5,1986,1986-02-02,Wild Coast Class,0.00000,7,2,Sunshine Tour,Afr,True
1,2,1986,1986-01-12,Tourn. of Champ.,116.00000,1,23,PGA Tour,PGAT,True
2,5,1986,1986-02-02,Tasmanian Open,0.00000,8,4,PGA Tour of Australasia,ANZ,True
3,4,1986,1986-01-26,Lexington PGA,0.00000,5,2,Sunshine Tour,Afr,True
4,3,1986,1986-01-19,ICL Internat'nal,0.00000,3,2,Sunshine Tour,Afr,True
...,...,...,...,...,...,...,...,...,...,...
29,25,2024,2024-06-23,Nashville_liv,NaN,32_liv,41,Liv Golf,liv,False
30,28,2024,2024-07-14,Andalucía_liv,NaN,33_liv,41,Liv Golf,liv,False
31,30,2024,2024-07-28,United Kingdom_liv,NaN,34_liv,41,Liv Golf,liv,False
32,33,2024,2024-08-18,Greenbrier _liv,NaN,35_liv,41,Liv Golf,liv,False


In [195]:
merged_df = pd.merge(df_combined, events, on="tournament_id", how="left")
merged_df

,player_id,name,position,points_won,start_rank,end_rank,tournament_id,R1,R2,R3,...,total,week,year,date,event,field_rating,tour_id,tour_name,tour_code,is_eligible
0,449,Mark McNulty,1,10.0,NaN,NaN,7,NaN,NaN,NaN,...,NaN,5,1986,1986-02-02,Wild Coast Class,0.00000,2,Sunshine Tour,Afr,True
1,440,David Frost,2,6.0,NaN,NaN,7,NaN,NaN,NaN,...,NaN,5,1986,1986-02-02,Wild Coast Class,0.00000,2,Sunshine Tour,Afr,True
2,446,Tony Johnstone,3,4.0,NaN,NaN,7,NaN,NaN,NaN,...,NaN,5,1986,1986-02-02,Wild Coast Class,0.00000,2,Sunshine Tour,Afr,True
3,95,Rick Hartmann,T4,4.0,NaN,NaN,7,NaN,NaN,NaN,...,NaN,5,1986,1986-02-02,Wild Coast Class,0.00000,2,Sunshine Tour,Afr,True
4,265,David Feherty,T4,4.0,NaN,NaN,7,NaN,NaN,NaN,...,NaN,5,1986,1986-02-02,Wild Coast Class,0.00000,2,Sunshine Tour,Afr,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1140530,19455,Guillermo Mito Pereira,T47,NaN,NaN,NaN,36_liv,73.0,71.0,71.0,...,215.0,37,2024,2024-09-15,Chicago_liv,NaN,41,Liv Golf,liv,False
1140531,12337,Jason Kokrak,51,NaN,NaN,NaN,36_liv,73.0,73.0,70.0,...,216.0,37,2024,2024-09-15,Chicago_liv,NaN,41,Liv Golf,liv,False
1140532,15856,Cameron Smith,52,NaN,NaN,NaN,36_liv,73.0,73.0,74.0,...,220.0,37,2024,2024-09-15,Chicago_liv,NaN,41,Liv Golf,liv,False
1140533,14706,Hudson Swafford,53,NaN,NaN,NaN,36_liv,78.0,78.0,67.0,...,223.0,37,2024,2024-09-15,Chicago_liv,NaN,41,Liv Golf,liv,False


In [196]:
merged_df.to_csv("project_data.csv", index=False)

In [178]:
metadata

,Week,Event Date,Event Title,Field Rating,tournament_id,tourId,currencyId,name,fullName,code,isEligible,date,week,year
0,5,2ND FEBRUARY 1986,Wild Coast Class,0.00000,7,2,0,Sunshine Tour,Sunshine Tour,Afr,True,1986-02-02,5,1986
1,2,12TH JANUARY 1986,Tourn. of Champ.,116.00000,1,23,0,PGA Tour,PGA Tour,PGAT,True,1986-01-12,2,1986
2,5,2ND FEBRUARY 1986,Tasmanian Open,0.00000,8,4,0,PGA Tour of Australasia,PGA Tour of Australasia,ANZ,True,1986-02-02,5,1986
3,4,26TH JANUARY 1986,Lexington PGA,0.00000,5,2,0,Sunshine Tour,Sunshine Tour,Afr,True,1986-01-26,4,1986
4,3,19TH JANUARY 1986,ICL Internat'nal,0.00000,3,2,0,Sunshine Tour,Sunshine Tour,Afr,True,1986-01-19,3,1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9149,40,6TH OCTOBER 2024,Vizag Open,4.99762,10606,25,0,Professional Golf Tour of India,Professional Golf Tour of India,PGTI,True,2024-10-06,40,2024
9150,40,6TH OCTOBER 2024,Ryo Ishikawa everyone PROJECT Challenge,7.23574,10604,7,0,Abema TV Tour,Abema TV Tour,ATVT,True,2024-10-06,40,2024
9151,40,6TH OCTOBER 2024,ACN Championship Golf Tournament,33.52272,10603,15,0,Japan Golf Tour,Japan Golf Tour,Jpn,True,2024-10-06,40,2024
9152,40,6TH OCTOBER 2024,Mercuries Taiwan Masters,24.52166,10605,5,0,Asian Tour,Asian Tour,Asa,True,2024-10-06,40,2024


In [179]:
metadata["date"] = pd.to_datetime(metadata["Event Date"])
metadata["date"] = pd.to_datetime(metadata["date"])

metadata["week"] = metadata["date"].dt.isocalendar().week
metadata["year"] = metadata["date"].dt.isocalendar().year

metadata["date"] = metadata["date"].dt.date

# metadata['week'] = metadata['Week'].str.extract('(\d+)').astype(int)
metadata

/var/folders/76/xwx2gdv102jgvj2y2l_m0n8w0000gn/T/ipykernel_1658/2347864032.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  metadata["date"] = pd.to_datetime(metadata["Event Date"])


,Week,Event Date,Event Title,Field Rating,tournament_id,tourId,currencyId,name,fullName,code,isEligible,date,week,year
0,5,2ND FEBRUARY 1986,Wild Coast Class,0.00000,7,2,0,Sunshine Tour,Sunshine Tour,Afr,True,1986-02-02,5,1986
1,2,12TH JANUARY 1986,Tourn. of Champ.,116.00000,1,23,0,PGA Tour,PGA Tour,PGAT,True,1986-01-12,2,1986
2,5,2ND FEBRUARY 1986,Tasmanian Open,0.00000,8,4,0,PGA Tour of Australasia,PGA Tour of Australasia,ANZ,True,1986-02-02,5,1986
3,4,26TH JANUARY 1986,Lexington PGA,0.00000,5,2,0,Sunshine Tour,Sunshine Tour,Afr,True,1986-01-26,4,1986
4,3,19TH JANUARY 1986,ICL Internat'nal,0.00000,3,2,0,Sunshine Tour,Sunshine Tour,Afr,True,1986-01-19,3,1986
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9149,40,6TH OCTOBER 2024,Vizag Open,4.99762,10606,25,0,Professional Golf Tour of India,Professional Golf Tour of India,PGTI,True,2024-10-06,40,2024
9150,40,6TH OCTOBER 2024,Ryo Ishikawa everyone PROJECT Challenge,7.23574,10604,7,0,Abema TV Tour,Abema TV Tour,ATVT,True,2024-10-06,40,2024
9151,40,6TH OCTOBER 2024,ACN Championship Golf Tournament,33.52272,10603,15,0,Japan Golf Tour,Japan Golf Tour,Jpn,True,2024-10-06,40,2024
9152,40,6TH OCTOBER 2024,Mercuries Taiwan Masters,24.52166,10605,5,0,Asian Tour,Asian Tour,Asa,True,2024-10-06,40,2024


/var/folders/76/xwx2gdv102jgvj2y2l_m0n8w0000gn/T/ipykernel_1658/2274815483.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  metadata["date"] = pd.to_datetime(metadata["Event Date"])


,week,year,date,event,field_rating,tournament_id,tourId,name,code,isEligible
0,5,1986,1986-02-02,Wild Coast Class,0.00000,7,2,Sunshine Tour,Afr,True
1,2,1986,1986-01-12,Tourn. of Champ.,116.00000,1,23,PGA Tour,PGAT,True
2,5,1986,1986-02-02,Tasmanian Open,0.00000,8,4,PGA Tour of Australasia,ANZ,True
3,4,1986,1986-01-26,Lexington PGA,0.00000,5,2,Sunshine Tour,Afr,True
4,3,1986,1986-01-19,ICL Internat'nal,0.00000,3,2,Sunshine Tour,Afr,True
...,...,...,...,...,...,...,...,...,...,...
29,25,2024,2024-06-23,Nashville_liv,NaN,32_liv,41,Liv Golf,liv,False
30,28,2024,2024-07-14,Andalucía_liv,NaN,33_liv,41,Liv Golf,liv,False
31,30,2024,2024-07-28,United Kingdom_liv,NaN,34_liv,41,Liv Golf,liv,False
32,33,2024,2024-08-18,Greenbrier _liv,NaN,35_liv,41,Liv Golf,liv,False


In [182]:
events.tourId.max()

np.float64(40.0)

In [149]:
events.to_csv("proejct_events.csv", index=False)